In [1]:
import pandas as pd

In [2]:
index_data = pd.read_excel('D:\\rqadatas\\000001.xlsx', 
                           parse_dates=['时间'], 
                           date_parser=lambda date:date.split(',')[0])

In [3]:
index_data.head()

,时间,开盘,最高,最低,收盘,涨跌,涨幅%,振幅%,总手(万),金额(亿)
0,1990-12-19,96.05,99.98,95.79,99.98,0.00,0.00,0.00,49400,494311
1,1990-12-20,104.30,104.39,99.98,104.39,4.41,4.41,4.41,8400,84992
2,1990-12-21,109.07,109.13,103.73,109.13,4.74,4.54,5.17,1600,16096
3,1990-12-24,113.57,114.55,109.13,114.55,5.42,4.97,4.97,3100,31063
4,1990-12-25,120.09,120.25,114.55,120.25,5.70,4.98,4.98,600,6510


In [4]:
index_data.columns

Index(['时间', '开盘', '最高', '最低', '收盘', '涨跌', '涨幅%', '振幅%', '总手(万)', '金额(亿)'], dtype='object')

In [27]:
index_data = index_data[['时间','最高','最低','收盘','涨幅%']]

In [28]:
index_data.sort_values('时间', inplace=True)

In [29]:
N1=20
N2=10

In [30]:
index_data['最近%s个交易日的最高点' % N1] = index_data['最高'].rolling(N1, min_periods=1).max()

In [9]:
index_data['最近%s个交易日的最低点' % N2] = index_data['最低'].rolling(N2, min_periods=1).min()

In [10]:
buy_index = index_data[index_data['收盘'] > index_data['最近%s个交易日的最高点' % N1].shift(1)].index

In [11]:
index_data.loc[buy_index, '收盘时发出的信号'] = 1

In [12]:
sell_index = index_data[index_data['收盘'] < index_data['最近%s个交易日的最低点' % N2].shift(1)].index

In [13]:
index_data.loc[sell_index, '收盘时发出的信号'] = 0

In [14]:
index_data['当天的仓位'] = index_data['收盘时发出的信号'].shift(1)

In [15]:
index_data['当天的仓位'].fillna(method='ffill', inplace=True)

In [16]:
index_data = index_data[index_data['时间'] >= pd.to_datetime('19930101')]

In [17]:
index_data['资金指数']=(index_data['涨幅%'] / 100 * index_data['当天的仓位'] + 1).cumprod()

In [18]:
imitial_idx = index_data.iloc[0]['收盘'] / (1 + index_data.iloc[0]['涨幅%'] / 100)

In [20]:
index_data['资金指数'] *= imitial_idx

In [22]:
#index_data.to_csv('d:\\rqadatas\\turtle_result.cvs', index=False, encoding='utf8')

In [21]:
index_data['海龟法则每日涨跌幅度'] = index_data['涨幅%'] / 100 * index_data['当天的仓位']

In [23]:
index_data['涨幅'] = index_data['涨幅%'] / 100

In [24]:
year_rtn = index_data.set_index('时间')[['涨幅','海龟法则每日涨跌幅度']].resample('A').\
apply(lambda x:(x + 1.0).prod() - 1.0) * 100

In [40]:
year_rtn


,涨幅,海龟法则每日涨跌幅度
时间,,
1993-12-31,6.863311,47.468796
1994-12-31,-22.347255,74.793525
1995-12-31,-14.306219,-14.214456
1996-12-31,65.270111,22.307939
1997-12-31,30.211569,16.019608
1998-12-31,-3.990290,7.356947
1999-12-31,19.259991,19.239058
2000-12-31,51.705328,18.603585
2001-12-31,-20.529087,3.179190
